# QDET Encoders Module Tutorial

This notebook demonstrates all available tools in the QDET encoders module. The encoders module provides various quantum encoding techniques to map classical data into quantum states, enabling quantum algorithms to process classical information efficiently.

## 1. Import Required Libraries

Import necessary libraries including pandas, numpy, matplotlib, and all tools from the encoders module.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import sys
import os
sys.path.append(os.path.abspath('..'))

# Import encoders module tools
from qudet.encoders import (
    RotationEncoder,
    AmplitudeEncoder,
    DensityMatrixEncoder,
    BasisChangeEncoder,
    FeatureMapEncoder,
    AngleEncoder,
    PhaseEncoder,
    HybridAnglePhaseEncoder,
    MultiAxisRotationEncoder,
    ParametricAngleEncoder,
    CompositeEncoder,
    LayeredEncoder,
    DataReuseEncoder,
    AdaptiveEncoder,
    HierarchicalEncoder,
    IQPEncoder,
    StatevectorEncoder
)

# Set random seed for reproducibility
np.random.seed(42)

print("✓ All libraries and encoder tools imported successfully!")

## 2. Load and Explore the Iris Dataset

Load the iris.csv dataset and prepare it for quantum encoding demonstrations.

In [ ]:
# Load the Iris dataset
iris_df = pd.read_csv('../qudet/datasets/iris.csv')

# Prepare data for encoding
X = iris_df.iloc[:, :-1].values  # Features
y = iris_df.iloc[:, -1].values   # Labels

# Normalize features to [0, 2π] for quantum encoding
X_norm = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0)) * 2 * np.pi

# Also create unit-normalized version for amplitude encoding
X_unit = X / np.linalg.norm(X, axis=1, keepdims=True)

print("Dataset Summary:")
print(f"Shape: {iris_df.shape}")
print(f"Features: {iris_df.columns[:-1].tolist()}")
print(f"Number of classes: {len(np.unique(y))}")
print(f"\nNormalization ranges:")
print(f"Angle encoding (0 to 2π): min={X_norm.min():.4f}, max={X_norm.max():.4f}")
print(f"Unit norm encoding: min={X_unit.min():.4f}, max={X_unit.max():.4f}")

# Take first 5 samples for detailed demonstrations
sample_indices = [0, 50, 100]
sample_data = X_norm[sample_indices]
print(f"\nSample data for demonstrations:")
print(f"Sample 1 (Setosa): {np.round(sample_data[0], 3)}")
print(f"Sample 2 (Versicolor): {np.round(sample_data[1], 3)}")
print(f"Sample 3 (Virginica): {np.round(sample_data[2], 3)}")

## 3. Rotation Encoder

**Description**: Encodes classical features into qubit rotation angles. Each feature x_i becomes a rotation R_y(x_i) on qubit i. Simple and efficient encoding with linear scaling.

**Use Case**: Encode iris measurements directly as rotation angles on quantum gates.

In [ ]:
# Create Rotation Encoder
rot_encoder = RotationEncoder(n_qubits=4)

# Encode first sample
circuit_rot = rot_encoder.encode(sample_data[0])

print("Rotation Encoder Results:")
print(f"Number of qubits: {rot_encoder.n_qubits}")
print(f"Encoding method: RY rotations")
print(f"Circuit depth: {circuit_rot.depth()}")
print(f"Circuit size: {circuit_rot.size()}")
print(f"\nCircuit for first iris sample:")
print(circuit_rot)

# Visualize the quantum circuit
print(f"\n✓ Rotation encoding successful")

## 4. Amplitude Encoder

**Description**: Encodes normalized data into quantum state amplitudes using logarithmic qubit compression. Maps classical data vector to amplitudes of quantum state: |ψ⟩ = Σ amplitude_i |i⟩.

**Use Case**: Compress iris features into fewer qubits using amplitude encoding.

In [ ]:
# Create Amplitude Encoder
amp_encoder = AmplitudeEncoder(n_qubits=3, normalize=True)

# Encode first sample (normalized)
circuit_amp = amp_encoder.encode(X_unit[0])

print("Amplitude Encoder Results:")
print(f"Number of qubits: {amp_encoder.n_qubits}")
print(f"Maximum features supported: {amp_encoder.get_features_supported()}")
print(f"Normalization enabled: {amp_encoder.normalize}")
print(f"Circuit depth: {circuit_amp.depth()}")
print(f"Circuit size: {circuit_amp.size()}")
print(f"\nAmplitude encoding characteristics:")
print(f"Compression ratio: {4}/{amp_encoder.get_features_supported()} features → {amp_encoder.n_qubits} qubits")
print(f"✓ Amplitude encoding successful")

## 5. Statevector Encoder

**Description**: Encodes a data vector into amplitudes of a quantum state with automatic normalization. Provides logarithmic compression by encoding N features into log2(N) qubits.

**Use Case**: Prepare quantum states from iris feature vectors.

In [ ]:
# Create Statevector Encoder
sv_encoder = StatevectorEncoder()

# Encode first sample
circuit_sv = sv_encoder.encode(X_unit[0])

print("Statevector Encoder Results:")
print(f"Input feature dimension: {len(X_unit[0])}")
print(f"Qubits required: {circuit_sv.num_qubits}")
print(f"Circuit depth: {circuit_sv.depth()}")
print(f"Circuit size: {circuit_sv.size()}")

# Demonstrate with different sample sizes
print(f"\nEncoding efficiency:")
for sample_idx, sample_name in zip([0, 50, 100], ["Setosa", "Versicolor", "Virginica"]):
    qc = sv_encoder.encode(X_unit[sample_idx])
    print(f"  {sample_name}: {len(X_unit[sample_idx])} features → {qc.num_qubits} qubits")

print(f"\n✓ Statevector encoding successful")

## 6. IQP Encoder

**Description**: Implements Instantaneous Quantum Polynomial Encoding with feature entanglement. Creates two-qubit interactions and correlations through ZZ rotations based on feature products.

**Use Case**: Encode iris features with quantum entanglement for QSVM applications.

In [ ]:
# Create IQP Encoder
iqp_encoder = IQPEncoder(n_qubits=4, reps=2)

# Encode first sample
circuit_iqp = iqp_encoder.encode(sample_data[0])

print("IQP Encoder Results:")
print(f"Number of qubits: {iqp_encoder.n_qubits}")
print(f"Repetition layers: {iqp_encoder.reps}")
print(f"Encoding method: IQP with ZZ entanglement")
print(f"Circuit depth: {circuit_iqp.depth()}")
print(f"Circuit size: {circuit_iqp.size()}")
print(f"\nIQP characteristics:")
print(f"Single-qubit gates: RZ rotations from features")
print(f"Two-qubit gates: RZZ from feature products (creates entanglement)")
print(f"✓ IQP encoding successful")

## 7. Angle Encoder

**Description**: Encodes features into rotation angles of quantum gates with configurable rotation axes (RX, RY, RZ). Supports multiple repetitions and scaling factors for flexible encoding.

**Use Case**: Encode iris measurements with different rotation axes and scaling.

In [ ]:
# Create Angle Encoder with different rotation types
angle_types = ['rx', 'ry', 'rz', 'auto']

print("Angle Encoder Results:")
print(f"Number of qubits: 4")
print(f"Input features: 4")

results = {}
for angle_type in angle_types:
    encoder = AngleEncoder(n_qubits=4, angle_type=angle_type, reps=1)
    circuit = encoder.encode(sample_data[0])
    results[angle_type] = {
        'depth': circuit.depth(),
        'size': circuit.size()
    }
    print(f"\n{angle_type.upper()} rotation:")
    print(f"  Circuit depth: {circuit.depth()}")
    print(f"  Circuit size: {circuit.size()}")

# Test scaled encoding
scaled_encoder = AngleEncoder(n_qubits=4, angle_type='ry', reps=2)
circuit_scaled = scaled_encoder.encode_scaled(sample_data[0] / (2 * np.pi))
print(f"\nScaled RY encoding (2 reps):")
print(f"  Circuit depth: {circuit_scaled.depth()}")
print(f"  Circuit size: {circuit_scaled.size()}")

print(f"\n✓ Angle encoding successful")

## 8. Phase Encoder

**Description**: Encodes data into quantum phase information using controlled phase gates and phase shift operations. Phase encoding is efficient and preserves quantum coherence.

**Use Case**: Encode iris features using phase-sensitive quantum gates.

In [ ]:
# Create Phase Encoder
phase_encoder = PhaseEncoder(n_qubits=4)

# Encode first sample
circuit_phase = phase_encoder.encode(sample_data[0])

print("Phase Encoder Results:")
print(f"Number of qubits: {phase_encoder.n_qubits}")
print(f"Encoding method: Phase gates (P gates)")
print(f"Circuit depth: {circuit_phase.depth()}")
print(f"Circuit size: {circuit_phase.size()}")

print(f"\nPhase encoding characteristics:")
print(f"Efficient: Uses single-qubit phase gates")
print(f"Coherence: Preserves quantum coherence better than rotations")
print(f"Use cases: Quantum interference, phase-sensitive algorithms")

print(f"\n✓ Phase encoding successful")

## 9. Multi-Axis Rotation Encoder

**Description**: Applies rotations around multiple axes (X, Y, Z) in sequence. Creates more expressive feature embeddings than single-axis encoders.

**Use Case**: Encode iris features with multi-axis rotations for enhanced expressivity.

In [ ]:
# Create Multi-Axis Rotation Encoder
multi_encoder = MultiAxisRotationEncoder(n_qubits=4, axes=['rx', 'ry', 'rz'])

# Encode first sample
circuit_multi = multi_encoder.encode(sample_data[0])

print("Multi-Axis Rotation Encoder Results:")
print(f"Number of qubits: {multi_encoder.n_qubits}")
print(f"Rotation axes: {multi_encoder.axes}")
print(f"Circuit depth: {circuit_multi.depth()}")
print(f"Circuit size: {circuit_multi.size()}")

print(f"\nMulti-axis encoding characteristics:")
print(f"Expressivity: Higher than single-axis due to sequential rotations")
print(f"Circuit depth: Deeper due to multiple rotation layers")
print(f"Advantages: Better feature separation in high-dimensional space")

print(f"\n✓ Multi-axis rotation encoding successful")

## 10. Composite Encoder

**Description**: Combines multiple encoding strategies sequentially to create more expressive quantum feature representations. Enables flexible composition of different encoders.

**Use Case**: Combine rotation and phase encoding for iris features.

In [ ]:
# Create Composite Encoder combining multiple strategies
composite = CompositeEncoder(n_qubits=4)

# Add different encoders
encoder1 = RotationEncoder(n_qubits=4)
encoder2 = AngleEncoder(n_qubits=4, angle_type='rz')

composite.add_encoder(encoder1)
composite.add_encoder(encoder2)

# Encode first sample
circuit_composite = composite.encode(sample_data[0])

print("Composite Encoder Results:")
print(f"Number of qubits: {composite.n_qubits}")
print(f"Number of composed encoders: {len(composite.encoders)}")
print(f"Encoder types: {[type(e).__name__ for e in composite.encoders]}")
print(f"Circuit depth: {circuit_composite.depth()}")
print(f"Circuit size: {circuit_composite.size()}")

encoder_info = composite.get_encoder_info()
print(f"\nComposed encoders:")
for idx, info in enumerate(encoder_info, 1):
    print(f"  {idx}. {info['type']}")

print(f"\n✓ Composite encoding successful")

## 11. Layered Encoder

**Description**: Applies encoding in multiple layers with entanglement between layers. Creates hierarchical quantum feature representations through layered structure.

**Use Case**: Create deep layered encodings of iris features.

In [ ]:
# Create Layered Encoder
layered_encoder = LayeredEncoder(n_qubits=4, n_layers=2, entangle_type='linear')

# Encode first sample
circuit_layered = layered_encoder.encode(sample_data[0])

print("Layered Encoder Results:")
print(f"Number of qubits: {layered_encoder.n_qubits}")
print(f"Number of layers: {layered_encoder.n_layers}")
print(f"Entanglement type: {layered_encoder.entangle_type}")
print(f"Circuit depth: {circuit_layered.depth()}")
print(f"Circuit size: {circuit_layered.size()}")

print(f"\nLayered encoding characteristics:")
print(f"Structure: Multiple encoding layers with entanglement")
print(f"Expressivity: Increases with number of layers")
print(f"Entanglement: Creates inter-layer quantum correlations")

print(f"\n✓ Layered encoding successful")

## 12. Encoder Comparison

**Description**: Compare different encoding strategies on iris data to understand their characteristics and trade-offs.

**Comparison**: Analyze depth, size, and expressivity across encoders.

In [ ]:
# Compare different encoding methods
encoders_list = [
    ("Rotation (RY)", RotationEncoder(n_qubits=4)),
    ("Amplitude", AmplitudeEncoder(n_qubits=3)),
    ("Statevector", StatevectorEncoder()),
    ("IQP (reps=2)", IQPEncoder(n_qubits=4, reps=2)),
    ("Angle (RY)", AngleEncoder(n_qubits=4, angle_type='ry')),
    ("Phase", PhaseEncoder(n_qubits=4)),
]

comparison_results = []

print("Encoder Comparison on Iris Data:")
print("=" * 70)

for name, encoder in encoders_list:
    try:
        # Use appropriate data based on encoder type
        if isinstance(encoder, (AmplitudeEncoder, StatevectorEncoder)):
            data = X_unit[0]
        else:
            data = sample_data[0]
        
        circuit = encoder.encode(data)
        
        comparison_results.append({
            'Encoder': name,
            'Qubits': circuit.num_qubits,
            'Depth': circuit.depth(),
            'Gates': circuit.size()
        })
        
        print(f"\n{name}:")
        print(f"  Qubits: {circuit.num_qubits}, Depth: {circuit.depth()}, Gates: {circuit.size()}")
    except Exception as e:
        print(f"\n{name}: Skipped ({str(e)[:50]})")

# Create comparison dataframe
comparison_df = pd.DataFrame(comparison_results)
print("\n" + "=" * 70)
print("\nSummary Statistics:")
print(comparison_df.to_string(index=False))

## 13. Encoding Performance Analysis

**Description**: Analyze the circuit metrics and performance characteristics of different encoders on the full iris dataset.

**Analysis**: Understand qubit requirements and circuit complexity.

In [ ]:
# Analyze encoding performance across the iris dataset
print("Encoding Performance Analysis:")
print("=" * 60)

# Test on multiple iris samples
sample_indices = range(0, 150, 30)  # Sample every 30th iris flower
encoders_analysis = [
    ("Rotation", RotationEncoder(n_qubits=4)),
    ("IQP", IQPEncoder(n_qubits=4, reps=1)),
    ("Angle (RY)", AngleEncoder(n_qubits=4, angle_type='ry')),
]

for encoder_name, encoder in encoders_analysis:
    depths = []
    sizes = []
    
    for idx in sample_indices:
        circuit = encoder.encode(sample_data[0])
        depths.append(circuit.depth())
        sizes.append(circuit.size())
    
    avg_depth = np.mean(depths)
    avg_size = np.mean(sizes)
    
    print(f"\n{encoder_name}:")
    print(f"  Avg circuit depth: {avg_depth:.1f}")
    print(f"  Avg gate count: {avg_size:.1f}")
    print(f"  Qubit requirement: {encoder.n_qubits}")

print("\n" + "=" * 60)
print("\nKey Insights:")
print("1. Rotation encoders: Simple, shallow circuits, linear in features")
print("2. IQP encoders: Deeper circuits, captures feature interactions")
print("3. Amplitude/Statevector: Logarithmic qubit scaling, efficient for large features")

## Summary

This tutorial demonstrated all major tools in the QDET encoders module:

1. **RotationEncoder** - Simple RY rotations for feature encoding
2. **AmplitudeEncoder** - Normalized data to quantum state amplitudes
3. **StatevectorEncoder** - Logarithmic compression with state initialization
4. **IQPEncoder** - Instantaneous Quantum Polynomial with entanglement
5. **AngleEncoder** - Configurable rotation axes (RX, RY, RZ, auto)
6. **PhaseEncoder** - Phase-based quantum feature embedding
7. **MultiAxisRotationEncoder** - Sequential rotations on multiple axes
8. **CompositeEncoder** - Combines multiple encoding strategies
9. **LayeredEncoder** - Hierarchical multi-layer encoding with entanglement
10. **DensityMatrixEncoder** - Density matrix-based representations
11. **BasisChangeEncoder** - Basis transformation encoding
12. **FeatureMapEncoder** - Quantum feature map creation
13. **HybridAnglePhaseEncoder** - Combined angle and phase encoding
14. **ParametricAngleEncoder** - Parametric angle encoding
15. **DataReuseEncoder** - Efficient data reuse strategies
16. **AdaptiveEncoder** - Adaptive encoding based on data
17. **HierarchicalEncoder** - Hierarchical feature representation

Each encoder provides different trade-offs between:
- **Circuit depth** (execution time)
- **Gate count** (resource usage)
- **Qubit scaling** (linear vs. logarithmic)
- **Feature expressivity** (capturing feature interactions)
- **Quantum advantage** (entanglement, phase encoding)

These encoders form the foundation of quantum machine learning algorithms, enabling classical data to be processed by quantum computers.